# Preprocess the data 

In [1]:
import pandas as pd
import numpy as np
import gzip
import csv

In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

lim1 = 10000

def getDF(path):
    i = 0
    df = {}
    
    for d in parse(path):
        df[i] = d
        i += 1
        if i == lim1:
            break
    return pd.DataFrame.from_dict(df, orient='index')

The model consists of:

- 0 for beauty
- 1 for phone

In [ ]:
# for preprocessing

#source
df_beauty = getDF('reviews_Beauty_5.json.gz')
df_phone = getDF('reviews_Cell_Phones_and_Accessories_5.json.gz')

#target
df_health = getDF('reviews_Health_and_Personal_Care_5.json.gz')
df_apps = getDF('reviews_Apps_for_Android_5.json.gz')

In [71]:
#target 2
lim1 = 9000

def getDF(path):
    i = 0
    df = {}
    
    for d in parse(path):
        df[i] = d
        i += 1
        if i == lim1:
            break
    return pd.DataFrame.from_dict(df, orient='index')

df_tools = getDF('reviews_Tools_and_Home_Improvement_5.json.gz')
df_food = getDF('reviews_Grocery_and_Gourmet_Food_5.json.gz')

print(df_beauty.head())

       reviewerID        asin reviewerName helpful  \
0  A1YJEY40YUW4SE  7806397051       Andrea  [3, 4]   
1   A60XNB876KYML  7806397051   Jessica H.  [1, 1]   
2  A3G6XNM240RMWA  7806397051        Karen  [0, 1]   
3  A1PQFP6SAJ6D80  7806397051        Norah  [2, 2]   
4  A38FVHZTNQ271F  7806397051    Nova Amor  [0, 0]   

                                          reviewText  overall  \
0  Very oily and creamy. Not at all what I expect...      1.0   
1  This palette was a decent price and I was look...      3.0   
2  The texture of this concealer pallet is fantas...      4.0   
3  I really can't tell what exactly this thing is...      2.0   
4  It was a little smaller than I expected, but t...      3.0   

                  summary  unixReviewTime   reviewTime  
0  Don't waste your money      1391040000  01 30, 2014  
1             OK Palette!      1397779200  04 18, 2014  
2           great quality      1378425600   09 6, 2013  
3  Do not work on my face      1386460800   12 8, 2013  

In [5]:
# source domain

# column 1
c1_1 = df_beauty['reviewText']
c1_2 = df_phone['reviewText']
c1 = c1_1.append(c1_2, ignore_index=True)


# column 2
c2_1 = pd.Series(np.zeros(df_beauty.shape[0], dtype=int))
c2_2 = pd.Series(np.ones(df_phone.shape[0], dtype=int))
c2 = c2_1.append(c2_2, ignore_index=True)

df = pd.DataFrame({'label': c2, 'features': c1})
df.to_csv('rev_pre.csv', index=False)


with open("rev_pre.csv",'r') as f:
    with open("rev_source.csv",'w') as f1:
        i = 0
        for line in f:
            if i:
                f1.write(line)
            i = 1

In [60]:
# target domain

# column 1
c1_1 = df_health['reviewText']
c1_2 = df_apps['reviewText']
c1 = c1_1.append(c1_2, ignore_index=True)


# column 2
c2_1 = pd.Series(np.zeros(df_health.shape[0], dtype=int))
c2_2 = pd.Series(np.ones(df_apps.shape[0], dtype=int))
c2 = c2_1.append(c2_2, ignore_index=True)

df = pd.DataFrame({'label': c2, 'features': c1})
df.to_csv('rev_pre.csv', index=False)


with open("rev_pre.csv",'r') as f:
    with open("rev_target1.csv",'w') as f1:
        i = 0
        for line in f:
            if i:
                f1.write(line)
            i = 1

In [72]:
# target domain 2

# column 1
c1_1 = df_tools['reviewText']
c1_2 = df_food['reviewText']
c1 = c1_1.append(c1_2, ignore_index=True)


# column 2
c2_1 = pd.Series(np.zeros(df_tools.shape[0], dtype=int))
c2_2 = pd.Series(np.ones(df_food.shape[0], dtype=int))
c2 = c2_1.append(c2_2, ignore_index=True)

df = pd.DataFrame({'label': c2, 'features': c1})
df.to_csv('rev_pre.csv', index=False)


with open("rev_pre.csv",'r') as f:
    with open("rev_target2.csv",'w') as f1:
        i = 0
        for line in f:
            if i:
                f1.write(line)
            i = 1

# Load the data 

In [10]:
import torch
import random
from torch import nn
from torchtext import data

assert torch.__version__ == '1.0.0'

In [14]:
# source

FEATURES_FREE = data.Field(lower=True)
LABEL_FREE = data.Field(sequential=False, is_target=True, unk_token=None)
#FEATURES_FREE = data.Field(tokenize=lambda s: s.split(" "))

rt_polarity_free = data.TabularDataset(
    path=r'C:\Users\luiss\Documents\1ScriptsPython\NLP\rev_source.csv', format='csv',
    fields=[('label', LABEL_FREE), ('features', FEATURES_FREE)])

FEATURES_FREE.build_vocab(rt_polarity_free)
LABEL_FREE.build_vocab(rt_polarity_free)


# Show one example
print(rt_polarity_free[1].features)
print(rt_polarity_free[1].label)

print(rt_polarity_free[-1].features)
print(rt_polarity_free[-1].label)

['this', 'palette', 'was', 'a', 'decent', 'price', 'and', 'i', 'was', 'looking', 'for', 'a', 'few', 'different', 'shades.', 'this', 'palette', 'conceals', 'decently,', 'however,', 'it', 'does', 'somewhat', 'cake', 'up', 'and', 'crease.']
0
['good', 'style', 'is', 'essential', 'to', 'any', 'apple', 'design.', 'i', 'have', 'never', 'understood', 'why', 'third', 'party', 'vendors', 'have', 'offered', 'cases', 'for', 'the', 'iphone', '3g', 'and', '3gs', 'which', 'look', 'like', 'they', 'were', 'designed', 'by', 'an', 'ad', 'hoc', 'committee', 'at', 'the', 'lenin', 'tankworks', 'in', 'minsk.here', 'we', 'have', 'an', 'exception.', 'the', 'mophie', 'case', 'is', 'simple,', 'elegant', 'and', 'completely', 'complementary', 'to', "apple's", 'styling.', 'but', "likewrigley's", '5', 'stick', 'doublemint', 'gum', '40', 'packs,', '"it\'s', 'two,', 'two..."', '-two', 'devices', 'in', 'one.', 'it', 'has', 'a', 'powerful', 'external', 'battery', 'that', 'extends', 'the', 'notoriously', 'battery', 'lik

In [15]:
print(len(FEATURES_FREE.vocab), len(LABEL_FREE.vocab))

86674 2


# Set the model 

In [16]:
PAD_INDEX = 1

# Model
class LogisticRegression(nn.Module):
    """
    Simple Logistic Regression implementation based on torchtext input format.
    """
    def __init__(self, num_features):
        super(LogisticRegression, self).__init__()
        self.weights = nn.Parameter(torch.normal(torch.zeros(num_features)), requires_grad=True)
            
    def forward(self, x):
        """
        Args:
          x: a batch of input text of shape [max_sentence_length, batch_size] using 1 for padding. 
        """
        
        # retrieve weights and set those to zero that come from padding cells 
        active_tokens_mask = (x != PAD_INDEX).float()
        #print(active_tokens_mask.shape)
        #print(x.shape)
        #print(self.weights.shape)
        filtered = active_tokens_mask * self.weights[x]
        
        # sum pooling along the token position dimension 
        logits = filtered.sum(0)
        return logits

# Code for the training loop 

In [94]:
def accuracy(dataset, model, batch_size=32):
    # Testing the model and returning the accuracy on the given dataset
    total = 0
    correct = 0
    for batch in data.BucketIterator(dataset=dataset, batch_size=batch_size):
        output = model(batch.features)
        total += len(batch.label)
        prediction = (output > 0).long()
        correct += (prediction == batch.label).sum()

    return float(correct) / total  

def training_loop(model, train_set, dev_set, num_epochs=100, 
                  batch_size=32, lr=0.1, weight_decay=0.0, print_val=100):
    """
    Should return the best dev_set accuracy and the number of epochs used. 
    """
    criterion = torch.nn.BCEWithLogitsLoss()  
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, 
                              weight_decay=weight_decay)  
    # Training the Model
    epoch_accuracies = []
    epoch_acc_list = []
    best_epoch = 0
    best_accuracy = 0.0
    for epoch in range(num_epochs):
        for i, batch in enumerate(data.BucketIterator(dataset=train_set, batch_size=batch_size)):
            features = batch.features
            labels = batch.label.float()
            #print(features)
            #print(labels)

            # Forward + Backward + Optimize
            optimizer.zero_grad()
            outputs = model(features)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if (i+1) % print_val == 0:
                print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f, Dev: %.4f' 
                     % (epoch+1, num_epochs, i+1, len(train_set)//batch_size, 
                        loss.data, accuracy(dev_set, model)))

        epoch_accuracies.append(accuracy(dev_set, model))
        epoch_acc_list.append(accuracy(dev_set, model))
        if epoch_accuracies[-1] > best_accuracy:
            best_accuracy = epoch_accuracies[-1]
            best_epoch = epoch

        # TODO: early stopping here  
        if epoch>10:
            if epoch_accuracies[-11] >= epoch_accuracies[-1]:
                break
    
    return epoch_acc_list

# Run the model 

In [107]:
# initialise the data

random.seed(10)

rt_train_free, rt_dev_free = rt_polarity_free.split([0.7, 0.3], random_state=random.getstate())

# TODO: implement grid search to make sure the above 4 variable have the right values 
train_set = rt_train_free
dev_set = rt_dev_free

print(len(train_set))
print(len(dev_set))

14000
6000


In [146]:
ilr = 1.0
iw = 0.0001

print('\n \t lr: ',ilr, 'weigth_decay: ',iw)

model_source = LogisticRegression(len(FEATURES_FREE.vocab))

epoch_list = training_loop(model_source, train_set, dev_set, 
                                       num_epochs=10, batch_size=32,
                                       lr=ilr, weight_decay=iw, print_val = 200)
print(epoch_list)


 	 lr:  1.0 weigth_decay:  0.0001
Epoch: [1/10], Step: [200/437], Loss: 0.0867, Dev: 0.8693
Epoch: [1/10], Step: [400/437], Loss: 0.0219, Dev: 0.9018
Epoch: [2/10], Step: [200/437], Loss: 0.0145, Dev: 0.9137
Epoch: [2/10], Step: [400/437], Loss: 0.0231, Dev: 0.9210
Epoch: [3/10], Step: [200/437], Loss: 0.0112, Dev: 0.9250
Epoch: [3/10], Step: [400/437], Loss: 0.0163, Dev: 0.9328
Epoch: [4/10], Step: [200/437], Loss: 0.0126, Dev: 0.9332
Epoch: [4/10], Step: [400/437], Loss: 0.0160, Dev: 0.9390
Epoch: [5/10], Step: [200/437], Loss: 0.0150, Dev: 0.9377
Epoch: [5/10], Step: [400/437], Loss: 0.0152, Dev: 0.9415
Epoch: [6/10], Step: [200/437], Loss: 0.0174, Dev: 0.9402
Epoch: [6/10], Step: [400/437], Loss: 0.0128, Dev: 0.9437
Epoch: [7/10], Step: [200/437], Loss: 0.0186, Dev: 0.9427
Epoch: [7/10], Step: [400/437], Loss: 0.0113, Dev: 0.9452
Epoch: [8/10], Step: [200/437], Loss: 0.0191, Dev: 0.9443
Epoch: [8/10], Step: [400/437], Loss: 0.0108, Dev: 0.9475
Epoch: [9/10], Step: [200/437], Loss:

# Transfer 1 - similar

In [110]:
# target

FEATURES_TAR = data.Field(lower=True)
LABEL_TAR = data.Field(sequential=False, is_target=True, unk_token=None)

rt_polarity_tar = data.TabularDataset(
    path=r'C:\Users\luiss\Documents\1ScriptsPython\NLP\rev_target1.csv', format='csv',
    fields=[('label', LABEL_TAR), ('features', FEATURES_TAR)])

FEATURES_TAR.build_vocab(rt_polarity_tar)
LABEL_TAR.build_vocab(rt_polarity_tar)


# Show one example
print(rt_polarity_tar[1].features)
print(rt_polarity_tar[1].label)

print(rt_polarity_tar[-1].features)
print(rt_polarity_tar[-1].label)

['i', 'would', 'recommend', 'this', 'for', 'a', 'travel', 'magnifier', 'for', 'the', 'occasional', 'reading.i', 'had', 'read', 'on', 'another', 'review', 'about', 'a', 'magnifier', 'having', 'a', 'problem', 'with', 'the', 'light', 'coming', 'on.', 'i', 'did', 'find', 'that', 'this', 'one', 'appeared', 'to', 'be', 'doa', 'out', 'of', 'the', 'box.', 'but,', 'after', 'opening', '&', 'shutting', 'the', 'viewer', 'to', 'turn', 'on', '&', 'off', 'the', 'light,', 'the', 'light', 'began', 'to', 'come', 'on.', 'after', 'several', 'times', 'of', 'doing', 'this,', 'the', 'light', 'appears', 'to', 'be', 'coming', 'on', 'all', 'the', 'time.it', 'is', 'small,', 'but', 'for', 'taking', 'it', 'someplace', '&', 'reading', 'things', 'like', 'a', 'menu', 'in', 'a', 'dark', 'corner', 'of', 'a', 'restaurant,', 'this', 'is', 'great.']
0
['this', 'is', 'a', 'great', 'app.', "i'm", 'doing', 'the', 'leangains', 'diet', 'and', 'this', 'app', 'makes', 'calorie', 'counting', 'very', 'simple', 'and', 'accessible.'

In [111]:
print(len(FEATURES_TAR.vocab), len(LABEL_TAR.vocab))

69383 2


# Transfer 1 - run model in big set

In [112]:
# initialise the data

random.seed(10)

rt_train_tar, rt_dev_tar = rt_polarity_tar.split([0.7, 0.3], random_state=random.getstate())

# TODO: implement grid search to make sure the above 4 variable have the right values 
train_set_big = rt_train_tar
dev_set_big = rt_dev_tar

print(len(train_set))
print(len(dev_set))

14000
6000


In [113]:
ilr = 1.0
iw = 0.0001

print('\n \t lr: ',ilr, 'weigth_decay: ',iw)

model_tar = model_source

epoch_list = training_loop(model_tar, train_set_big, dev_set_big, 
                           num_epochs=10, batch_size=32,
                           lr=ilr, weight_decay=iw, print_val = 200)
print(epoch_list)


 	 lr:  1.0 weigth_decay:  0.0001
Epoch: [1/10], Step: [200/437], Loss: 0.1998, Dev: 0.8658
Epoch: [1/10], Step: [400/437], Loss: 0.4183, Dev: 0.9032
Epoch: [2/10], Step: [200/437], Loss: 0.1792, Dev: 0.9138
Epoch: [2/10], Step: [400/437], Loss: 0.3085, Dev: 0.9265
Epoch: [3/10], Step: [200/437], Loss: 0.1406, Dev: 0.9285
Epoch: [3/10], Step: [400/437], Loss: 0.2474, Dev: 0.9343
Epoch: [4/10], Step: [200/437], Loss: 0.1177, Dev: 0.9350
Epoch: [4/10], Step: [400/437], Loss: 0.1633, Dev: 0.9417
Epoch: [5/10], Step: [200/437], Loss: 0.1038, Dev: 0.9413
Epoch: [5/10], Step: [400/437], Loss: 0.1387, Dev: 0.9465
Epoch: [6/10], Step: [200/437], Loss: 0.0916, Dev: 0.9463
Epoch: [6/10], Step: [400/437], Loss: 0.1157, Dev: 0.9507
Epoch: [7/10], Step: [200/437], Loss: 0.0823, Dev: 0.9487
Epoch: [7/10], Step: [400/437], Loss: 0.0979, Dev: 0.9535
Epoch: [8/10], Step: [200/437], Loss: 0.0751, Dev: 0.9520
Epoch: [8/10], Step: [400/437], Loss: 0.0870, Dev: 0.9543
Epoch: [9/10], Step: [200/437], Loss:

# Transfer 1 - run model in small set

In [118]:
# initialise the data

random.seed(10)

#per_training_data = 0.1
#spliting_list = [1 - per_training_data, 0.7 * per_training_data, 0.3 * per_training_data]
#rt_surplus, rt_dev_tar, rt_train_tar = rt_polarity_tar.split(spliting_list, random_state=random.getstate())

per_training_data = 0.05
spliting_list = [per_training_data, 1 - per_training_data]
rt_train_tar, rt_dev_tar = rt_polarity_tar.split(spliting_list, random_state=random.getstate())


# TODO: implement grid search to make sure the above 4 variable have the right values 
train_set_small = rt_train_tar
dev_set_small = rt_dev_tar

print(len(train_set_small))
print(len(dev_set_small))

1000
19000


In [121]:
ilr = 1.0
iw = 0.0001

print('\n \t lr: ',ilr, 'weigth_decay: ',iw)

model_tar2 = model_source

epoch_list = training_loop(model_tar2, train_set_small, dev_set_small, 
                           num_epochs=10, batch_size=80,
                           lr=ilr, weight_decay=iw, print_val = 10)
print(epoch_list)


 	 lr:  1.0 weigth_decay:  0.0001
Epoch: [1/10], Step: [10/12], Loss: 2.4424, Dev: 0.5995
Epoch: [2/10], Step: [10/12], Loss: 1.8609, Dev: 0.6513
Epoch: [3/10], Step: [10/12], Loss: 1.4075, Dev: 0.6962
Epoch: [4/10], Step: [10/12], Loss: 1.0821, Dev: 0.7238
Epoch: [5/10], Step: [10/12], Loss: 0.8715, Dev: 0.7427
Epoch: [6/10], Step: [10/12], Loss: 0.7475, Dev: 0.7575
Epoch: [7/10], Step: [10/12], Loss: 0.6543, Dev: 0.7661
Epoch: [8/10], Step: [10/12], Loss: 0.5696, Dev: 0.7732
Epoch: [9/10], Step: [10/12], Loss: 0.5010, Dev: 0.7800
Epoch: [10/10], Step: [10/12], Loss: 0.4414, Dev: 0.7846
[0.6243684210526316, 0.6714736842105263, 0.7021578947368421, 0.7303157894736843, 0.748578947368421, 0.7519473684210526, 0.7598421052631579, 0.7692105263157895, 0.7757368421052632, 0.7832105263157895]


# Transfer 1 - run model in small set 2

In [147]:
# initialise the data

random.seed(10)

#per_training_data = 0.1
#spliting_list = [1 - per_training_data, 0.7 * per_training_data, 0.3 * per_training_data]
#rt_surplus, rt_dev_tar, rt_train_tar = rt_polarity_tar.split(spliting_list, random_state=random.getstate())

per_training_data = 0.5
spliting_list = [per_training_data, 1 - per_training_data]
rt_train_tar, rt_dev_tar = rt_polarity_tar.split(spliting_list, random_state=random.getstate())


# TODO: implement grid search to make sure the above 4 variable have the right values 
train_set_small = rt_train_tar
dev_set_small = rt_dev_tar

print(len(train_set_small))
print(len(dev_set_small))

10000
10000


In [148]:
ilr = 1.0
iw = 0.0001

print('\n \t lr: ',ilr, 'weigth_decay: ',iw)

model_tar2 = model_source

epoch_list = training_loop(model_tar2, train_set_small, dev_set_small, 
                           num_epochs=10, batch_size=80,
                           lr=ilr, weight_decay=iw, print_val = 100)
print(epoch_list)


 	 lr:  1.0 weigth_decay:  0.0001
[0.8333, 0.8771, 0.8983, 0.9073, 0.9134, 0.9186, 0.9229, 0.9269, 0.9298, 0.9342]


# Target 2 - different

In [122]:
# target 2

FEATURES_TAR2 = data.Field(lower=True)
LABEL_TAR2 = data.Field(sequential=False, is_target=True, unk_token=None)

rt_polarity_tar2 = data.TabularDataset(
    path=r'C:\Users\luiss\Documents\1ScriptsPython\NLP\rev_target2.csv', format='csv',
    fields=[('label', LABEL_TAR2), ('features', FEATURES_TAR2)])

FEATURES_TAR2.build_vocab(rt_polarity_tar2)
LABEL_TAR2.build_vocab(rt_polarity_tar2)

print(len(FEATURES_TAR2.vocab))

# Show one example
print(rt_polarity_tar2[1].features)
print(rt_polarity_tar2[1].label)

print(rt_polarity_tar2[-1].features)
print(rt_polarity_tar2[-1].label)

84158
['these', 'little', 'magnets', 'are', 'really', 'powerful', 'for', 'there', 'size.', 'i', 'am', 'using', 'them', 'to', 'make', 'secret', 'compartments', 'in', 'custom', 'made', 'boxes.', 'each', 'one', 'hols', 'about', '.8', 'of', 'a', 'pound.']
0
['this', 'is', 'very', 'good........i', 'add', 'a', 'little', 'ground', 'beef', 'sometimes.................good', 'either', 'way.very', 'easy', 'and', 'fast', 'to', 'prepare.for', 'some', 'reason,', 'it', 'is', 'hard', 'to', 'find', 'here;', 'so,', 'i', 'love', 'i', 'have', 'several', 'in', 'my', 'pantry.']
1


# Transfer 2 - run model in big set

In [129]:
# initialise the data

random.seed(10)

rt_train_tar2, rt_dev_tar2 = rt_polarity_tar2.split([0.7, 0.3], random_state=random.getstate())

# TODO: implement grid search to make sure the above 4 variable have the right values 
train_set_big = rt_train_tar2
dev_set_big = rt_dev_tar2

print(len(train_set_big))
print(len(dev_set_big))

12600
5400


In [130]:
ilr = 1.0
iw = 0.0001

print('\n \t lr: ',ilr, 'weigth_decay: ',iw)

model_tar2 = model_source

epoch_list = training_loop(model_tar2, train_set_big, dev_set_big, 
                           num_epochs=10, batch_size=80,
                           lr=ilr, weight_decay=iw, print_val = 100)
print(epoch_list)


 	 lr:  1.0 weigth_decay:  0.0001
Epoch: [1/10], Step: [100/157], Loss: 0.5837, Dev: 0.8256
Epoch: [2/10], Step: [100/157], Loss: 0.2602, Dev: 0.8907
Epoch: [3/10], Step: [100/157], Loss: 0.1646, Dev: 0.9120
Epoch: [4/10], Step: [100/157], Loss: 0.1230, Dev: 0.9224
Epoch: [5/10], Step: [100/157], Loss: 0.1014, Dev: 0.9300
Epoch: [6/10], Step: [100/157], Loss: 0.0916, Dev: 0.9348
Epoch: [7/10], Step: [100/157], Loss: 0.0812, Dev: 0.9393
Epoch: [8/10], Step: [100/157], Loss: 0.0745, Dev: 0.9422
Epoch: [9/10], Step: [100/157], Loss: 0.0688, Dev: 0.9446
Epoch: [10/10], Step: [100/157], Loss: 0.0639, Dev: 0.9461
[0.8657407407407407, 0.9016666666666666, 0.9162962962962963, 0.9268518518518518, 0.932962962962963, 0.9387037037037037, 0.9414814814814815, 0.9438888888888889, 0.9468518518518518, 0.9496296296296296]


# Transfer 2 - run model in small set

In [137]:
# initialise the data

random.seed(10)

#per_training_data = 0.1
#spliting_list = [1 - per_training_data, 0.7 * per_training_data, 0.3 * per_training_data]
#rt_surplus, rt_dev_tar, rt_train_tar = rt_polarity_tar.split(spliting_list, random_state=random.getstate())

per_training_data = 0.05
spliting_list = [per_training_data, 1 - per_training_data]
rt_train_tar2, rt_dev_tar2 = rt_polarity_tar2.split(spliting_list, random_state=random.getstate())


# TODO: implement grid search to make sure the above 4 variable have the right values 
train_set_small = rt_train_tar2
dev_set_small = rt_dev_tar2

print(len(train_set_small))
print(len(dev_set_small))

900
17100


In [138]:
ilr = 1.0
iw = 0.0001

print('\n \t lr: ',ilr, 'weigth_decay: ',iw)

model_tar2 = model_source

epoch_list = training_loop(model_tar2, train_set_small, dev_set_small, 
                           num_epochs=10, batch_size=80,
                           lr=ilr, weight_decay=iw, print_val = 10)
print(epoch_list)


 	 lr:  1.0 weigth_decay:  0.0001
Epoch: [1/10], Step: [10/11], Loss: 2.2825, Dev: 0.6104
Epoch: [2/10], Step: [10/11], Loss: 5.1326, Dev: 0.6694
Epoch: [3/10], Step: [10/11], Loss: 2.3636, Dev: 0.7104
Epoch: [4/10], Step: [10/11], Loss: 0.7029, Dev: 0.7373
Epoch: [5/10], Step: [10/11], Loss: 0.5786, Dev: 0.7555
Epoch: [6/10], Step: [10/11], Loss: 0.5849, Dev: 0.7325
Epoch: [7/10], Step: [10/11], Loss: 0.4156, Dev: 0.7825
Epoch: [8/10], Step: [10/11], Loss: 0.3261, Dev: 0.7879
Epoch: [9/10], Step: [10/11], Loss: 0.2761, Dev: 0.7916
Epoch: [10/10], Step: [10/11], Loss: 0.2545, Dev: 0.7968
[0.5593567251461988, 0.5385380116959064, 0.6192397660818714, 0.7287719298245614, 0.7575438596491229, 0.5522807017543859, 0.7821637426900585, 0.7871929824561403, 0.79046783625731, 0.7947953216374269]
